In [ ]:
import mitsuba as mi
mi.set_variant("llvm_ad_rgb")
import drjit as dr

import numpy as np
import matplotlib.pyplot as plt
import cmap_diff
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

import time
import os
base_dir = 'estimator_comparison_veach'
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
    
mi.Thread.thread().logger().set_log_level(mi.LogLevel.Warn)

In [ ]:
scene = mi.load_file('scenes/veach.xml')
params = mi.traverse(scene)
param_key = 'plates_top.bsdf.alpha.data'
params.keep([param_key])

roughness_tex = mi.TensorXf(params[param_key])

diff_methods = [
    'es_detached',
    
    'bs_detached',
    'bs_attached',
    
    'mis_detached_detached',
    'mis_attached_attached',
    'mis_detached_attached',
    'mis_attached_detached',
    
    'bs_detached_diff',
    'mis_detached_detached_diff',
    
    'bs_attached_reparam',
    'mis_attached_attached_reparam',
]
diff_method_names = [
    'Detached emitter sampling',
    
    'Detached BSDF sampling',
    'Attached BSDF sampling (BIASED!)',
    
    'Detached MIS weights, detached BSDF sampling,\ndetached emitter sampling',
    'Attached MIS weights, attached BSDF sampling,\ndetached emitter sampling (BIASED!)',
    'Detached MIS weights, attached BSDF sampling,\ndetached emitter sampling (BIASED!)',
    'Attached MIS weights, detached BSDF sampling,\ndetached emitter sampling',
    
    'Detached diff. BSDF sampling',
    'Detached MIS weights, detached diff. BSDF sampling,\ndetached emitter sampling',
    
    'Attached reparam. BSDF sampling',
    'Attached MIS weights, attached reparam. BSDF sampling,\ndetached emitter sampling'
]
diff_method_spps = [
    128,
    
    128,
    128,
    
    128,
    128,
    128,
    128,
    
    128,
    128,
    
    64,
    64,
]

In [ ]:
# Render a converged primal image and a finite differences gradient image for comparison
# (This might take a while ...)
eps = 1e-3
fd_spp = 2048

integrator = mi.load_dict({'type': 'path', 'max_depth': 2})
image = mi.render(scene, params, integrator=integrator, seed=0, spp=fd_spp)
outname = "{}/primal.exr".format(base_dir)
mi.util.convert_to_bitmap(image, uint8_srgb=False).write(outname)

roughness_tex_fd = roughness_tex + eps
params[param_key] = roughness_tex_fd
params.update()

image_fd = mi.render(scene, params, integrator=integrator, seed=0, spp=fd_spp)
outname = "{}/grad_fd.exr".format(base_dir)
mi.util.convert_to_bitmap((image_fd - image) / eps, uint8_srgb=False).write(outname)

In [ ]:
for method, method_name, spp in zip(diff_methods, diff_method_names, diff_method_spps):
    print("* {}".format(method_name))
    print("  spp = {}".format(spp))
    integrator = mi.load_dict({'type': 'estimator_comparison',
                               'method': method,
                               'reparam_kappa': 1e5,
                               'reparam_rays': 32})
    
    # Diff. input parameter
    pi = mi.Float(0.0)
    dr.enable_grad(pi)
    dr.set_grad(pi, 1.0)
    params[param_key] = roughness_tex + pi
    params.update()
    
    start = time.time()
    
    image_grad = mi.Float(0.0)
    if 'diff' in method:
        # Differential sampling strategy, use antithetic sampling.
        # Note that we use the same seed twice, and use half the number of samples for each pass.
        image = mi.render(scene, params,
                          integrator=integrator, seed=0, spp=spp//2, antithetic_pass=False)
        dr.forward(pi)
        image_grad = dr.grad(image)
    
        params[param_key] = roughness_tex + pi
        params.update()

        image = mi.render(scene, params,
                          integrator=integrator, seed=0, spp=spp//2, antithetic_pass=True)
        dr.forward(pi)
        image_grad += dr.grad(image)

        # Average both passes
        image_grad *= 0.5
    else:
        # Produce differentiable rendering
        image = mi.render(scene, params, integrator=integrator, seed=0, spp=spp)
        # And propagate derivatives forwards through it
        dr.forward(pi)
        image_grad = dr.grad(image)
        
    # Save output gradient image
    outname = "{}/grad_{}.exr".format(base_dir, method)
    mi.util.convert_to_bitmap(image_grad, uint8_srgb=False).write(outname)
    
    end = time.time()    
    print("  took {:.2f} seconds".format(end - start))

In [ ]:
# Display images and save PNGs
image_primal = np.array(mi.Bitmap("{}/primal.exr".format(base_dir)))
image_primal = np.clip(image_primal**(1/2.2), 0.0, 1.0) # Crude gamma correction
plt.imsave('{}/primal.jpg'.format(base_dir), image_primal)

plt.figure()
plt.imshow(image_primal)
plt.axis('off')
plt.title("Primal")
plt.show()

image_fd = np.array(mi.Bitmap("{}/grad_fd.exr".format(base_dir)))[:, :, 0]
plt.imsave('{}/grad_fd.jpg'.format(base_dir), image_fd, vmin=-20, vmax=+20, cmap='diff')

plt.figure()
plt.imshow(image_fd, cmap='diff', vmin=-20, vmax=+20)
plt.axis('off')
plt.title("Finite differences")
plt.show()

for method, method_name in zip(diff_methods, diff_method_names):
    image_grad = np.array(mi.Bitmap("{}/grad_{}.exr".format(base_dir, method)))[:, :, 0]
    plt.imsave('{}/grad_{}.jpg'.format(base_dir, method), image_grad, vmin=-20, vmax=+20, cmap='diff')
    
    plt.figure()
    plt.imshow(image_grad, cmap='diff', vmin=-20, vmax=+20)
    plt.axis('off')
    plt.title(method_name, size=7)
    plt.show()